In [51]:
using JuMP
using GLPK

In [52]:
m = Model();
set_optimizer(m, GLPK.Optimizer);

In [53]:
set_time_limit_sec(m, 1200.0);

In [54]:
function reader(filepath :: String)
  k_count = 0
  arestas = []
  v_count = 0
  
  open(filepath, "r") do f
    line = readline(f)
    line = readline(f)
    v_count = parse(Int, line) # Number of vertices
    line = readline(f)
    line = readline(f)

    while ! eof(f)
      line = readline(f)
      words = split(line)
      if isempty(words)
          continue
      end
      vertex1 = parse(Int,words[1])
      vertex2 = parse(Int,words[2])
      edge_type = parse(Int,words[3])

      if edge_type > k_count
        k_count = edge_type
      end

      push!(arestas, (vertex1,vertex2,edge_type))
    end
  end
  return (arestas, k_count, v_count)
end

reader (generic function with 1 method)

In [55]:
filepath = "/home/leo/Documents/2021_1/Otimizacao/Trabalho/final-otimizacao/rm/RM01"

my_tuple = reader(filepath)
arestas = my_tuple[1]
K = my_tuple[2]
V = my_tuple[3];

In [56]:
function create_arestas_k(arestas, k)
    arestas_k = [[] for i=1:k]
    for a in arestas
        push!(arestas_k[a[3]], (a))
    end
    return arestas_k
end

create_arestas_k (generic function with 1 method)

In [57]:
arestas_k = create_arestas_k(arestas,K);

In [58]:
@variable(m, x[i in arestas], Bin);

In [59]:
@constraint(m, [k in 1:K], sum(x[i] for i in arestas_k[k]) <=1);

In [60]:
function create_arestas_v(arestas, v)
    arestas_v = [[] for i=1:v]
    for a in arestas
        push!(arestas_v[a[1]], (a))
        push!(arestas_v[a[2]], (a))
    end
    return arestas_v
end

create_arestas_v (generic function with 1 method)

In [61]:
arestas_v = create_arestas_v(arestas,V);

In [62]:
@constraint(m, [v in 1:V], sum(x[i] for i in arestas_v[v]) <=1);

In [63]:
@objective(m, Max, sum(x[i] for i in arestas));

In [64]:
optimize!(m)

In [65]:
objective_value(m)

LoadError: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.

In [66]:
@show value.(x)

LoadError: Result index of attribute MathOptInterface.VariablePrimal(1) out of bounds. There are currently 0 solution(s) in the model.